In [1]:
import os
import pandas as pd
import numpy as np
import re
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Path to the dataset (adjust if running locally or in a different environment)
data_dir = "/kaggle/input/satellite-image-caption-generation/"

# Read CSV files
train_df = pd.read_csv(os.path.join(data_dir, "train.csv"))
val_df   = pd.read_csv(os.path.join(data_dir, "valid.csv"))
test_df  = pd.read_csv(os.path.join(data_dir, "test.csv"))

# If the CSV has multiple captions per image in one row, flatten them into individual rows
def flatten_captions(df):
    """If dataframe has a column with multiple captions (list or semicolon-separated), flatten into individual rows."""
    if 'captions' in df.columns:
        # If captions are in a single column as list or separated string
        captions_arr = []
        filepaths_arr = []
        for idx, row in df.iterrows():
            caps = row['captions']
            # If it's a string of list (e.g. "['cap1', 'cap2', ...]"), evaluate to list
            if isinstance(caps, str) and caps.startswith('[') and caps.endswith(']'):
                try:
                    caps_list = eval(caps)  # convert string repr of list to actual list
                except:
                    caps_list = [caps]  # if eval fails, treat entire string as one caption
            elif isinstance(caps, str) and ';' in caps:
                # If captions are semicolon-separated string
                caps_list = [c.strip() for c in caps.split(';') if c.strip()]
            elif isinstance(caps, list):
                caps_list = caps
            else:
                caps_list = [str(caps)]
            for cap in caps_list:
                captions_arr.append(cap)
                # If filepath or filename column exists, use it; else use an 'image' or similar column
                if 'filepath' in df.columns:
                    filepaths_arr.append(row['filepath'])
                elif 'filename' in df.columns:
                    filepaths_arr.append(row['filename'])
                else:
                    filepaths_arr.append(row[0])  # fallback to first column as image path
        new_df = pd.DataFrame({'filepath': filepaths_arr, 'caption': captions_arr})
        return new_df
    else:
        # If multiple caption columns (like caption1...caption5)
        caption_cols = [col for col in df.columns if col.lower().startswith('caption')]
        if caption_cols:
            captions_arr = []
            filepaths_arr = []
            for idx, row in df.iterrows():
                for col in caption_cols:
                    cap = str(row[col])
                    if cap in ("nan", "None") or len(cap.strip()) == 0:
                        continue  # skip empty
                    captions_arr.append(cap)
                    fp_col = 'filepath' if 'filepath' in row.index else 'filename' if 'filename' in row.index else None
                    if fp_col:
                        filepaths_arr.append(row[fp_col])
                    else:
                        filepaths_arr.append(row[0])
            new_df = pd.DataFrame({'filepath': filepaths_arr, 'caption': captions_arr})
            return new_df
    # If no complex structure, just rename caption column to a standard name if present
    if 'caption' in df.columns:
        return df.rename(columns={'caption': 'caption'})
    elif 'Caption' in df.columns:
        return df.rename(columns={'Caption': 'caption'})
    else:
        # Already one caption per row
        df_copy = df.copy()
        df_copy['caption'] = df_copy[df_copy.columns[-1]]  # assume last column is caption text
        return df_copy

# Apply flattening if needed
train_data = flatten_captions(train_df)
val_data   = flatten_captions(val_df)
test_data  = flatten_captions(test_df)

# Preview the number of samples
print(f"Training samples: {len(train_data)}; Validation samples: {len(val_data)}; Test samples: {len(test_data)}")

# Clean and tokenize captions
def clean_text(sentence):
    # Lowercase and remove non-alphanumeric characters (except spaces)
    sentence = sentence.lower()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    sentence = sentence.strip()
    return sentence

train_data['caption'] = train_data['caption'].astype(str).apply(clean_text)
val_data['caption']   = val_data['caption'].astype(str).apply(clean_text)
test_data['caption']  = test_data['caption'].astype(str).apply(clean_text)

# Build vocabulary from training captions
from collections import Counter
word_counter = Counter()
for caption in train_data['caption']:
    word_counter.update(caption.split())

# You can set a frequency threshold to filter rare words (optional)
freq_threshold = 1  # include all words that appear at least once
words = [word for word, cnt in word_counter.items() if cnt >= freq_threshold]
# Add special tokens
special_tokens = ['<pad>', '<unk>', '<end>']  # (We'll use image as implicit <start>)
vocab_words = special_tokens + sorted(words)  # sorted for consistency (optional)
# Create word2idx and idx2word dicts
word2idx = {word: idx for idx, word in enumerate(vocab_words)}
idx2word = {idx: word for idx, word in enumerate(vocab_words)}

vocab_size = len(word2idx)
pad_idx = word2idx['<pad>']
unk_idx = word2idx['<unk>']
end_idx = word2idx['<end>']

print(f"Vocabulary size: {vocab_size} (including pad/unk/end tokens)")

# Define a custom Dataset
class ImageCaptionDataset(Dataset):
    def __init__(self, df, images_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.images_dir = images_dir  # base directory containing images
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get image path and caption
        img_path = os.path.join(self.images_dir, self.df.loc[idx, 'filepath'])
        caption_text = str(self.df.loc[idx, 'caption'])
        # Open image
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # Tokenize caption to indices
        tokens = []
        for word in caption_text.split():
            tokens.append(word2idx.get(word, unk_idx))
        # Append <end> token
        tokens.append(end_idx)
        # Return image tensor and token list (not yet padded)
        return image, torch.tensor(tokens, dtype=torch.long)

# Define image transformation (ViT expects 224x224 images normalized to ImageNet stats)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Create Dataset instances for train, val, test
train_images_dir = data_dir
val_images_dir   = data_dir
test_images_dir  = data_dir

train_dataset = ImageCaptionDataset(train_data, train_images_dir, transform=transform)
val_dataset   = ImageCaptionDataset(val_data,   val_images_dir,   transform=transform)
test_dataset  = ImageCaptionDataset(test_data,  test_images_dir,  transform=transform)

# Define a collate function for DataLoader to pad sequences
def caption_collate_fn(batch):
    """
    Pads captions in the batch to the length of the longest caption and stacks images.
    """
    images, captions = zip(*batch)
    # Stack images (they are already tensors of same shape [3,224,224])
    images = torch.stack(images, dim=0)
    # Pad captions
    lengths = [cap.size(0) for cap in captions]
    max_len = max(lengths)
    padded_captions = torch.zeros(len(captions), max_len, dtype=torch.long) + pad_idx
    for i, cap in enumerate(captions):
        end = lengths[i]
        padded_captions[i, :end] = cap  # cap is already a tensor of indices
    return images, padded_captions

# Create DataLoaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=caption_collate_fn)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=caption_collate_fn)
test_loader  = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=caption_collate_fn)  # batch_size=1 for generation


Training samples: 8734; Validation samples: 1094; Test samples: 1093
Vocabulary size: 5100 (including pad/unk/end tokens)


In [2]:
import torch.nn as nn
import torchvision.models as models

# Define the Encoder-Decoder model
class ViTEncoderDecoder(nn.Module):
    def __init__(self, vocab_size, embed_dim=256, hidden_dim=512, num_layers=2, bidirectional=True):
        super(ViTEncoderDecoder, self).__init__()
        # Load a pretrained ViT model (ViT-B/16) from torchvision or timm
        # Here we use torchvision's ViT model for simplicity. If not available, timm could be used.
        self.encoder = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)  # pretrained weights
        # Remove/freeze the classification head so we get raw features
        # Torchvision ViT has encoder.heads (a Linear layer) for classification.
        # We replace it with identity to get the 768-dim embeddings.
        self.encoder.heads = nn.Identity()

        # Freeze ViT encoder parameters
        for param in self.encoder.parameters():
            param.requires_grad = False

        # Project image feature to decoder embedding size
        vit_output_dim = 768  # ViT-Base output feature dimension
        self.img_proj = nn.Linear(vit_output_dim, embed_dim)

        # Embedding layer for words
        self.word_embed = nn.Embedding(vocab_size, embed_dim, padding_idx=pad_idx)

        # LSTM decoder
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        num_directions = 2 if bidirectional else 1
        self.lstm = nn.LSTM(embed_dim, hidden_dim, num_layers=num_layers, 
                             bidirectional=bidirectional, batch_first=True)
        # Output layer to project LSTM hidden state to vocabulary
        self.fc_out = nn.Linear(hidden_dim * num_directions, vocab_size)

    def forward(self, images, captions):
        # Encode images with ViT (batch of images -> batch of feature vectors)
        # ViT expects input shape [batch, 3, 224, 224], outputs [batch, 768] after Identity head
        with torch.no_grad():
            img_features = self.encoder(images)  # shape: (batch, vit_output_dim)
        # Project image features to embedding size
        img_embeds = self.img_proj(img_features)  # shape: (batch, embed_dim)
        # Get word embeddings for captions
        # captions shape: (batch, seq_len)
        # We do not want to include the last token (<END>) as input, because we predict it.
        # So we take all but last token as input sequence for LSTM.
        captions_input = captions[:, :-1]  # remove last token for input
        word_embeds = self.word_embed(captions_input)  # shape: (batch, seq_len-1, embed_dim)
        # Prepend the image embedding to the sequence of word embeddings
        # img_embeds shape (batch, embed_dim) -> (batch, 1, embed_dim)
        img_embeds_seq = img_embeds.unsqueeze(1)  # add sequence dimension
        lstm_input = torch.cat([img_embeds_seq, word_embeds], dim=1)  # shape: (batch, seq_len, embed_dim), where seq_len = original caption length
        # Initialize LSTM state (h0, c0) as zeros
        # h0/c0 shape: (num_layers * num_directions, batch, hidden_dim)
        num_directions = 2 if self.bidirectional else 1
        h0 = torch.zeros(self.num_layers * num_directions, images.size(0), self.hidden_dim).to(images.device)
        c0 = torch.zeros(self.num_layers * num_directions, images.size(0), self.hidden_dim).to(images.device)
        # Pass through LSTM
        lstm_out, _ = self.lstm(lstm_input, (h0, c0))  # lstm_out: (batch, seq_len, hidden_dim*num_directions)
        # Compute output word scores
        outputs = self.fc_out(lstm_out)  # (batch, seq_len, vocab_size)
        return outputs

    def generate_caption(self, image, max_length=50):
        # Generate caption for a single image (image tensor of shape [3,224,224] or [1,3,224,224])
        self.eval()
        if image.ndim == 3:
            image = image.unsqueeze(0)  # add batch dim
        image = image.to(next(self.parameters()).device)
        # Encode image
        with torch.no_grad():
            img_feat = self.encoder(image)  # (1, vit_output_dim)
        img_embed = self.img_proj(img_feat)  # (1, embed_dim)
        # Initialize LSTM state
        num_directions = 2 if self.bidirectional else 1
        h = torch.zeros(self.num_layers * num_directions, 1, self.hidden_dim).to(image.device)
        c = torch.zeros(self.num_layers * num_directions, 1, self.hidden_dim).to(image.device)
        # Start generation with image embedding as first input
        input_token = img_embed  # shape (1, embed_dim)
        output_sentence = []
        for t in range(max_length):
            # LSTM expects input of shape (batch, seq_len, embed_dim), here (1,1,D)
            input_token = input_token.unsqueeze(1)  # (1,1,embed_dim)
            out, (h, c) = self.lstm(input_token, (h, c))  # out: (1,1, hidden_dim*num_directions)
            out = out.squeeze(1)  # (1, hidden*directions)
            scores = self.fc_out(out)  # (1, vocab_size)
            _, pred_idx = torch.max(scores, dim=1)  # predicted token index
            pred_idx = pred_idx.item()
            if pred_idx == end_idx or pred_idx == pad_idx:
                # Stop if end token is generated
                break
            output_word = idx2word.get(pred_idx, '<unk>')
            output_sentence.append(output_word)
            # Prepare next input token embedding (for next time step)
            input_token = self.word_embed(torch.tensor([pred_idx]).to(image.device))  # (1, embed_dim)
        return " ".join(output_sentence)

# Initialize model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ViTEncoderDecoder(vocab_size=vocab_size, embed_dim=256, hidden_dim=512, num_layers=2, bidirectional=True)
model = model.to(device)

criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


Downloading: "https://download.pytorch.org/models/vit_b_16-c867db91.pth" to /root/.cache/torch/hub/checkpoints/vit_b_16-c867db91.pth
100%|██████████| 330M/330M [00:01<00:00, 223MB/s]  


In [3]:
# Training the model
num_epochs = 5  # (You can increase this for better results; using 5 for brevity)
print_every = 100  # print training status every 100 batches

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    for batch_idx, (images, captions) in enumerate(train_loader):
        images, captions = images.to(device), captions.to(device)
        optimizer.zero_grad()
        # Forward pass
        outputs = model(images, captions)  # outputs: (batch, seq_len, vocab_size)

        targets = captions  # captions already have <END> at end and <PAD> beyond
        # Flatten outputs and targets for loss computation
        outputs_flat = outputs.view(-1, vocab_size)
        targets_flat = targets.view(-1)
        loss = criterion(outputs_flat, targets_flat)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (batch_idx+1) % print_every == 0:
            avg_loss = total_loss / (batch_idx+1)
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_loader)}], Avg Loss: {avg_loss:.4f}")
    avg_epoch_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} completed. Training Loss: {avg_epoch_loss:.4f}")


    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, captions in val_loader:
            images, captions = images.to(device), captions.to(device)
            outputs = model(images, captions)
            outputs_flat = outputs.view(-1, vocab_size)
            targets_flat = captions.view(-1)
            loss = criterion(outputs_flat, targets_flat)
            val_loss += loss.item()
    val_loss /= len(val_loader)
    print(f"Validation Loss after epoch {epoch+1}: {val_loss:.4f}\n")


Epoch [1/5], Batch [100/273], Avg Loss: 3.0293
Epoch [1/5], Batch [200/273], Avg Loss: 1.9365
Epoch 1 completed. Training Loss: 1.5415
Validation Loss after epoch 1: 0.6179

Epoch [2/5], Batch [100/273], Avg Loss: 0.2644
Epoch [2/5], Batch [200/273], Avg Loss: 0.2288
Epoch 2 completed. Training Loss: 0.2076
Validation Loss after epoch 2: 0.4131

Epoch [3/5], Batch [100/273], Avg Loss: 0.0749
Epoch [3/5], Batch [200/273], Avg Loss: 0.0715
Epoch 3 completed. Training Loss: 0.0699
Validation Loss after epoch 3: 0.3876

Epoch [4/5], Batch [100/273], Avg Loss: 0.0191
Epoch [4/5], Batch [200/273], Avg Loss: 0.0180
Epoch 4 completed. Training Loss: 0.0178
Validation Loss after epoch 4: 0.3785

Epoch [5/5], Batch [100/273], Avg Loss: 0.0036
Epoch [5/5], Batch [200/273], Avg Loss: 0.0032
Epoch 5 completed. Training Loss: 0.0030
Validation Loss after epoch 5: 0.3840



In [4]:
import matplotlib.pyplot as plt

model.eval()
# Pick a few samples from test set to visualize
num_samples_to_show = 5
samples = test_data.sample(num_samples_to_show)  # random sample of test DataFrame
fig, axes = plt.subplots(nrows=num_samples_to_show, ncols=1, figsize=(6, num_samples_to_show*4))
if num_samples_to_show == 1:
    axes = [axes]  # make it iterable

for ax, (_, sample) in zip(axes, samples.iterrows()):
    img_path = os.path.join(test_images_dir, sample['filepath'])
    # Open image (without transform) for display
    img = Image.open(img_path).convert('RGB')
    # Generate caption using the model
    img_tensor = transform(img)  # apply same transform (normed tensor) for the model
    img_tensor = img_tensor.to(device)
    predicted_caption = model.generate_caption(img_tensor, max_length=50)
    # Display
    ax.imshow(np.array(img))  # show the original image
    ax.set_title(f"Predicted: {predicted_caption}", fontsize=12)
    ax.axis('off')

plt.tight_layout()
plt.show()
